# MRE Assignment 3 - Digital Video Processing
In this assignment, you will use OpenCV and FFmpeg to implement very basic video editing functions. These tasks include:

1. Create a slide show (as a video) from images, and optionally create the slideshow as greyscale video.
2. Extract the audio track from a video file.
3. Replace the audio track in a video file.
4. Combine two or more videos into one video file.
5. Blend an image (fade-in/fade-out) with a video.
6. Blend two videos into one video (video collage).

In this notebook, you will implement your solution. This notebook will be imported into the "*_def.ipynb" notebook.

Of course you can include code for testing your implementation in this implementation notebook, but code for testing and output generated for testing is not going to be assessed.

Of course, your code for the solutions in this notebook will be inspected and is subject to grading.

## Setup

For general installation instructions, please refer to the ressources given for all the assignments [in Moodle](https://moodle.univie.ac.at/course/view.php?id=164140#section-12).

If the cell below executes without error, you can start the assignment!

In [1]:
# -------- Imports --------
# Please do not change the contents of this cell!

# Imports required by us.
import cv2              # opencv-python
import ffmpeg           # ffmpeg-python
import subprocess   # for calling local executables such as ffmpeg.exe
import pandas as pd  # pandas
from fractions import Fraction as frac # simplifying fractions

In the cells below, place your own imports, global variables, (helper) functions and classes. Feel free to add cells here as you see fit.

In [140]:
# Place your own imports here.
import IPython
from IPython.core.display import HTML

In [142]:
# Place any helper functions, global variables and classes here.

# For example:the function you need to play back a mp4-video file.
# You may use this function to display the videos in your definition file during the assessment for demoing the solutions.
def VideoPlayer(videoFile: str) -> None:
    IPython.display.display(
        HTML("""
            <video alt="test" controls>
            <source src={} type="video/mp4">
            </video>
        """.format(videoFile))
    )
    
    return

In [143]:
VideoPlayer("./resources/video/sample1.mp4")


## Task 3.1: Create a slide show (Video) from multiple images and convert it to greyscale 

In [158]:
# Write your function here.

# Replace the parameters and return type of the following function according to the task specification.
def CreateVideoFromImages(inImgLibFolder: str, imageFormat: str, durationInSec: int, convertToGreyScale: bool, outFolder: str, outVideo: str) -> None:
    if(convertToGreyScale):
        #pad=ceil(iw/2)*2:ceil(ih/2)*2, 
        subprocess.run("ffmpeg -y -framerate "+ str(durationInSec) +" -pattern_type glob -i '"+ inImgLibFolder +"*"+ imageFormat +"' -vf \"format=gray, scale=1280:720\" -c:v libx264 -pix_fmt yuv420p -shortest " + outFolder + outVideo + ".mp4", shell=True)
    else:
        subprocess.run("ffmpeg -y -framerate "+ str(durationInSec) +" -pattern_type glob -i '"+ inImgLibFolder +"*"+ imageFormat +"' -vf \"scale=1280:720\" -c:v libx264 -pix_fmt yuv420p -shortest " + outFolder + outVideo + ".mp4", shell=True)

    VideoPlayer(outFolder + outVideo + ".mp4")


In [160]:
# Test your function here.
CreateVideoFromImages("./resources/images/Task3.1/", ".jpg", 2, True, "./resources/images/Task3.1/", "slideshow")

## Task 3.2: Extract the Audio Track from a Video File 

In [ ]:
# Write your function here.

# Replace the parameters and return type of the following function according to the task specification.

def SplitAudioVideoTracks(...) -> None:
    pass


In [ ]:
# Test your function here.

## Task 3.3: Replace the Audio Track in a Video File

In [ ]:
# Write your function here.

# Replace the parameters and return type of the following function according to the task specification.

def AddOrReplaceAudio(...) -> None:
    pass

## Task 3.4: Combine Videos

In [ ]:
# Write your function here.

# Replace the parameters and return type of the following function according to the task specification.
def CombineVideos(...) -> None:
    pass

# Replace the parameters and return type of the following function according to the task specification.
def VideoMetadataExtractor(...) -> None:
    pass

In [ ]:
# Test your function here.

## Task 3.5: Blend an Image in a Video

In [ ]:
# Write your code here!

# Replace the parameters and return type of the following function according to the task specification.
def AddFadingImage(...) -> None: 
    pass

In [ ]:
# Test your function here.

## Task 3.6: Create a Video Collage - blend two videos into one

In [ ]:
# Write your function here.

# Replace the parameters and return type of the following function according to the task specification. 
def VideoClipMixer(...) -> None: 
    pass


In [ ]:
# Test your function here.